# 2. **File I/O**
In this tutorial, we'll explain how to read and download data.

In [ ]:
using SeisIO
include("safe_rm.jl")

## A. **Loading entire files**
`read_data` reads one or more entire files into memory.\
Use this with legacy data formats like SAC, SEG Y, and mini-SEED:

In [ ]:
?read_data

\
If you know the file format that you're trying to read, pass\
it as the first argument to `read_data` in lowercase:

In [ ]:
S = read_data("mseed", "DATA/2018.224.00.00.00.000.TA.C26K..BHZ.R.mseed")

In [ ]:
S = read_data("sac", "DATA/2018.224.00.00.00.000.TA.C26K..BHZ.R.SAC")

\
If you don't know a file's format, `read_data` calls a (somewhat slower)\
function called `guess`that can usually identify it:

In [ ]:
S2 = read_data("DATA/2018.224.00.00.00.000.TA.C26K..BHZ.R.SAC")

In [ ]:
S == S2

`read_data` accepts file string wildcards.

In [ ]:
path = dirname(pathof(SeisIO))*"/../test/SampleFiles/SUDS/"
S = read_data("sac", path * "*.sac")

## B. **Reading from large volumes**
Modern data formats create large volumes on disk, from which data are read in user-specified time windows.\
SeisIO currently supports reading from ASDF files through the function `read_hdf5`:

In [ ]:
?read_hdf5

In [ ]:
hdf = "DATA/2days-40hz.h5"
id  = "CI.SDD..HHZ"
ts  = "2019-07-07T23:00:00"
te  = "2019-07-08T01:59:59.975"
S = read_hdf5(hdf, ts, te, id = id)

\
FDSN-style wildcards can be used for the ID string.

In [ ]:
idr = "C*.SDD..HH?"
S1 = read_hdf5(hdf, ts, te, id = id)

In [ ]:
S == S1

\
If the `id` keyword isn't used, then all channels with data matching the time\
query are read into memory.

In [ ]:
S2 = read_hdf5(hdf, ts, te)

\
Contents of an HDF5 volume can be scanned at the "station" (default) or "trace" level with `scan_hdf5`:

In [ ]:
scan_hdf5(hdf)

In [ ]:
scan_hdf5(hdf, level="trace")

### File and Format Information (Optional Section)
Information on files and formats can be found in a number of places,\
including the command-line interface.

In [ ]:
guess("DATA/2018.224.00.00.00.000.TA.C26K..BHZ.R.SAC")

In [ ]:
path = dirname(pathof(SeisIO))*"/../test/SampleFiles/SUDS/"
fname = path * "lsm.sud"
g = guess(fname)

In [ ]:
SeisIO.formats[g[1]] # what are we looking at...?

In [ ]:
S2 = read_data(g[1], fname, swap = g[2])

In [ ]:
# since volcano colleagues keep asking
using SeisIO.SUDS
suds_support()

In [ ]:
# while I'm at it
using SeisIO.SEED

In [ ]:
?seed_support

In [ ]:
?mseed_support

...and knowing is half the battle.

## C. **Saving data**
SeisData and SeisChannel structures can be written to ASDF, SAC, or to SeisIO's\
native format. ASDF format is portable (most languages can read HDF5) and well-\
suited to large data sets. SAC has the advantage that it's almost universally\
readable, but only writes 32-bit float data, and each contiguous data segment\
creates one file. SeisIO format saves complete information and does low-level\
object writing with a file index; it's the fastest of the three writers.

### **Writing to ASDF files**
ASDF is an HDF5 file format designed for seismic data. Write to ASDF with the\
command `write_hdf5`. 

In [ ]:
?write_hdf5

#### **Creating ASDF volumes**
The simplest ASDF write creates a new file:

In [ ]:
hdf_out = "test.h5"
write_hdf5(hdf_out, S)

#### **Writing to existing ASDF volumes**
Writing to an existing ASDF volume adds the data to the `Waveforms/` group,\
creating new `Waveforms/(net.sta)/` paths as needed. Channel information is\
written to `Waveforms/(net.sta)/StationXML`:

In [ ]:
write_hdf5(hdf_out, S1)

#### **ASDF overwrite mode**
In "overwrite" mode (`write_hdf5(hdf_out, S, ovr=true)`), SeisIO only\
overwrites parts of traces in `hdf_out` that have sample times in `S`. As a\
demonstration, let's overwrite part of a channel's data with NaNs:

In [ ]:
S3 = S[1:1]
T = eltype(S3.x[1])
S3.t[1] = vcat(S3.t[1][1:1,:], [100 0])
S3.x[1] = T(NaN).*ones(100)
write_hdf5(hdf_out, S3, ovr=true)

\
Now we'll read back in and compare to `S[1]`:

In [ ]:
id = S3.id[1]
ts  = "2019-07-07T23:00:00"
te  = "2019-07-08T01:59:59.975"
S3 = read_hdf5(hdf_out, ts, te, id = id)

In [ ]:
minimum(isnan.(S3.x[1][1:100]))

In [ ]:
S3.x[1][101:end] == S.x[1][101:end]

#### **ASDF add mode**
In "add" mode (`write_hdf5(hdf_out, S, ovr=true)`), SeisIO overwrites parts of\
traces in `hdf_out` that have sample times in `S`, and adds new traces (filled\
with NaNs) corresponding to parts of `S` with no corresponding sample times in\
`hdf_out`. As a demonstration, let's create some new traces in `hdf_out`:

In [ ]:
using Dates
S2 = read_data("uw", "DATA/99011116541W")
write_hdf5(hdf_out, S2, add=true, len=Hour(1), chans=1:10, tag="raw")
scan_hdf5(hdf_out, level="trace")

\
Now let's read an example trace to see what we've created:

In [ ]:
id = S2.id[1]
ts = "1999-01-11T16:00:00"
te = "1999-01-11T17:59:59.99"
S3 = read_hdf5(hdf_out, ts, te, id = S2.id[1])

In [ ]:
i = findfirst(isnan.(S3.x[1]).==false)
X = S3.x[1][i:i-1+length(S2.x[1])]

In [ ]:
X == S2.x[1]

As the example shows, "add" mode creates placeholder traces; the intent is for\
data to written to these traces incrementally in "overwrite" mode.\
\
Let's clean this up and move on:

In [ ]:
safe_rm(hdf_out)

### **Writing to SAC files**
Writing to SAC creates one file for each contiguous data segment in each data\
channel. Although large collections of small files have been an obsolete data\
archival strategy since the early 2000s, SeisIO supports writing to SAC because\
the data format is almost universal.

In [ ]:
writesac(S)                         # filenames are auto-generated. no need to specify.
writesacpz(S, "req_1.pz")           # in case you need instrument responses later.

Two useful keywords control the creation of SAC files:
* `fname=FF` uses filename FF, rather than creating file names automatically. This keywork only works with GphysChannel objects.
* `xy=true` writes generic x-y data to file with time as the independent variable.

In [ ]:
writesac(S[1], fname="tmp.SAC")     # writesac also works with SeisChannel objects.

### **Writing to SeisIO files**
The SeisIO low-level file format is comprehensive and fast. A SeisIO file can\
contain multiple structures, and structures written to file can be any Type\
in SeisIO (including, for example, just an instrument response). Information\
in structures is fully preserved except for exotic data Types stored in the\
`:misc` field.\
\
To read from a SeisIO file, you'll need to specify one or more object numbers,\
or the output will be of Type `Array{Any,1}`:

In [ ]:
wseis("req_1.seis", S)
S2 = rseis("req_1.seis")[1]
S == S2

## D. **Further help**
Please consult the official SeisIO documentation:

### **Reading files with `read_data`**
https://seisio.readthedocs.io/en/latest/src/Formats/timeseries.html
 
### **Reading ASDF volumes with `read_hdf5`**
https://seisio.readthedocs.io/en/latest/src/Formats/hdf5.html
